# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
data_file = "../WeatherPy/city_list.csv"
data_file_df = pd.read_csv(data_file)
data_file_df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Sambava,97,MG,1597448796,84,-14.27,50.17,70.81,11.18
1,1,Vilcún,90,CL,1597448797,87,-38.65,-72.23,44.60,11.41
2,2,Zhuhai,40,CN,1597448797,79,22.28,113.57,84.20,4.70
3,3,Ilulissat,14,GL,1597448498,56,69.22,-51.10,42.80,3.36
4,4,Hermanus,45,ZA,1597448797,81,-34.42,19.23,55.99,13.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = data_file_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = data_file_df["Humidity"]

In [32]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
# Cities will be reduced based on the following weather condictions
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

reduced_cities_df = data_file_df.loc[(data_file_df["Wind Speed"] < 10) & (data_file_df["Cloudiness"] == 0) & \
                                   (data_file_df["Max Temp"] > 70) & (data_file_df["Max Temp"] < 80)].dropna()

reduced_cities_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
15,15,Kunduz,0,AF,1597448683,25,36.73,68.86,74.30,1.86
179,179,Saýat,0,TM,1597448829,38,38.78,63.88,77.00,4.70
234,234,Mehrān,0,IR,1597448842,27,33.12,46.16,73.40,2.24
250,250,Birjand,0,IR,1597448845,24,32.87,59.22,71.60,9.17
270,270,Damietta,0,EG,1597448850,83,31.42,31.81,77.97,7.61
292,292,Kiryat Gat,0,IL,1597448643,81,31.61,34.76,78.01,3.94
296,296,Coração de Jesus,0,BR,1597448854,45,-16.69,-44.37,70.16,8.37
410,410,Vila Velha,0,BR,1597448879,78,-20.33,-40.29,77.00,8.05
424,424,Colmenar Viejo,0,ES,1597448881,49,40.66,-3.77,75.99,5.01
494,494,Erzin,0,TR,1597448894,74,36.96,36.20,78.01,2.66


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df = reduced_cities_df.loc[:,["City", "Country", "Lat", "Lng"]]

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
15,Kunduz,AF,36.73,68.86,
179,Saýat,TM,38.78,63.88,
234,Mehrān,IR,33.12,46.16,
250,Birjand,IR,32.87,59.22,
270,Damietta,EG,31.42,31.81,
292,Kiryat Gat,IL,31.61,34.76,
296,Coração de Jesus,BR,-16.69,-44.37,
410,Vila Velha,BR,-20.33,-40.29,
424,Colmenar Viejo,ES,40.66,-3.77,
494,Erzin,TR,36.96,36.20,


In [36]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [41]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("No hotel")
        
    print("------------")


Retrieving Results for Index 15: Kunduz.
Closest hotel in Kunduz is عسکرشاه.
------------
Retrieving Results for Index 179: Saýat.
No hotel
------------
Retrieving Results for Index 234: Mehrān.
Closest hotel in Mehrān is پارکینگ شهید حججی.
------------
Retrieving Results for Index 250: Birjand.
Closest hotel in Birjand is Ghaem Hotel.
------------
Retrieving Results for Index 270: Damietta.
Closest hotel in Damietta is SULIMAN INN.
------------
Retrieving Results for Index 292: Kiryat Gat.
Closest hotel in Kiryat Gat is Desert Gat.
------------
Retrieving Results for Index 296: Coração de Jesus.
Closest hotel in Coração de Jesus is Millenium Hotel.
------------
Retrieving Results for Index 410: Vila Velha.
Closest hotel in Vila Velha is Hotel Senac Ilha do Boi.
------------
Retrieving Results for Index 424: Colmenar Viejo.
Closest hotel in Colmenar Viejo is Gcc Hotels Resorts Management Sl..
------------
Retrieving Results for Index 494: Erzin.
Closest hotel in Erzin is Hattusa Vacati

In [43]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
15,Kunduz,AF,36.73,68.86,عسکرشاه
179,Saýat,TM,38.78,63.88,
234,Mehrān,IR,33.12,46.16,پارکینگ شهید حججی
250,Birjand,IR,32.87,59.22,Ghaem Hotel
270,Damietta,EG,31.42,31.81,SULIMAN INN
292,Kiryat Gat,IL,31.61,34.76,Desert Gat
296,Coração de Jesus,BR,-16.69,-44.37,Millenium Hotel
410,Vila Velha,BR,-20.33,-40.29,Hotel Senac Ilha do Boi
424,Colmenar Viejo,ES,40.66,-3.77,Gcc Hotels Resorts Management Sl.
494,Erzin,TR,36.96,36.20,Hattusa Vacation Thermal Club Erzin


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))